In [1]:
import os
import sys
import torch

import numpy as np
import albumentations as album
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp

from torchmetrics import JaccardIndex
from torchmetrics.detection.mean_ap import MeanAveragePrecision

sys.path.insert(0, '..')
from loaders.datasets import AmsterdamDataset
from utils.general import visualize
from utils.augmentation import *

In [2]:
preprocessing_fn = smp.encoders.get_preprocessing_fn('resnet18', 'imagenet')
model = torch.load(os.path.join('..', 'models', 'weights'', 'best_model.pth'))

# images = os.path.join('..', 'data', 'images')
# annotations = os.path.join('..', 'data', 'spectrum-batch-1', 'annotations.json')

images = os.path.join('..', 'data', 'images')
annotations = os.path.join('..', 'data', 'fences-quays', 'annotations', 'test-annotations-6px.json')

data = AmsterdamDataset(images, annotations, preprocessing=get_preprocessing(preprocessing_fn))

FileNotFoundError: [Errno 2] No such file or directory: '..\\models\\cnns\\best_model.pth'

In [ ]:
%matplotlib inline

miou = 0
IoU = JaccardIndex(num_classes=2, reduction='none')
mAP = MeanAveragePrecision()

for i, (image, mask) in enumerate(data):
    mask = np.squeeze(mask)
    x = torch.as_tensor(image).unsqueeze(0).cuda()
    
    pred = model(x).detach().squeeze().cpu()#.numpy()
    image = np.transpose(image, (1, 2, 0))
    
    visualize(ground_truth=mask,
              prediction=pred)

    print(IoU(torch.as_tensor(pred).unsqueeze(0), torch.as_tensor(mask).unsqueeze(0).int()))
    miou += IoU(torch.as_tensor(pred).unsqueeze(0), torch.as_tensor(mask).unsqueeze(0).int())
    
    if i > 5:
        break
    
print(miou / (i + 1))